# Hyper Parameter Optimization using Talos

### Importing Necessary modules for the code

In [ ]:
# Install the plaidml backend
import plaidml.keras
plaidml.keras.install_backend()
import keras 
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.datasets import mnist
from keras.optimizers import RMSprop
import talos as ta
from livelossplot import PlotLossesKeras

%matplotlib inline
num_classes = 10
batch_size = 128
epochs = 10

### Importing data and splitting into training and test data 

#### The training data and test data is returned by the load data function from mnist. The number of training and test data are 60000

#### and 10000 respectively. These are stored in the variables 'x_training' and 'x_test'. The function load data returns two tuples of

#### arrays which are three dimensional numpy arrays of size (60000, 28, 28), 60000, (10000,28,28) and 10000. We will reshape these 

#### variables into our desired shape. 

In [ ]:
# the data is returned as a tuple of numpy arrays(data) which has a shape: x_training(60000,28,28),x_test(10000,28,28)
(x_training,y_training),(x_test, y_test) = mnist.load_data() 

# the shape of the array is converted as a two dimensional array of shape: x_training(60000,784),x_test(10000,784)
x_training = x_training.reshape(60000,784)
x_test = x_test.reshape(10000,784)

# The greyscale values range from 0 to 255 integers. We are interested in converting it into values between 0 and 1. 
x_training = x_training.astype('float32')
x_test = x_test.astype('float32')
x_training /= 255
x_test /= 255
print(x_training.shape[0], 'training sets')
print(x_test.shape[0], 'test sets')

### Converting class vectors in binary class matrices

In [ ]:
y_training = keras.utils.to_categorical(y_training, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

### Building the keras neural network model 

In [ ]:
def mnist_model(x_training, y_training, x_test, y_test, params):
    
    model = Sequential()

    model.add(Dense(params['first_neurons'], activation = 'relu', input_shape = (784,)))
    model.add(Dropout(0.2))
    model.add(Dense(512, activation = 'relu'))
    model.add(Dropout(0.2))
    model.add(Dense(num_classes, activation = 'softmax'))
    model.compile(loss = 'categorical_crossentropy', 
              optimizer = RMSprop(), 
              metrics = ['accuracy'])

    history = model.fit(x_training, y_training, 
                    batch_size = params['batch_size'], 
                    epochs = params['epochs'], 
                    verbose = 1,
                    #callbacks = [PlotLossesKeras()],
                    validation_data = (x_test, y_test), shuffle = True)

    return history, model


### Defining the parameter Space 

In [ ]:
p = {'first_neurons':[10,50,100,200,300,500,1000],
     'epochs': [50],
     'batch_size':[30]}

### Running the Experiment

In [ ]:
t = ta.Scan(x_training, y_training,
           params = p, model = mnist_model,
           dataset_name = 'mnist',
           experiment_no = '1',
            save_best_model = True,
            shuffle = True,
            search_method = 'random')